In [1]:
# Import necessary libraries

import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
# Spotify Web-API credentials

username = 'saminconso'
client_id ='2c36a6972d334e6e88ba8f6331b6efac'
client_secret = '45bf1d578b0c4e5f9056c3754d55e573'
redirect_uri = 'http://localhost:7777/callback'
#scope = 'user-read-recently-played'
scope = 'user-library-read'

In [3]:
# Generate authorization token

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [4]:
sp = spotipy.Spotify(auth=token)

In [15]:
# Process playlist and dump into dataframe

df_saved_tracks = pd.DataFrame()
track_list = ''
added_ts_list = []
artist_list = []
title_list = []
more_songs = True
offset_index = 0

while more_songs:
    songs = sp.current_user_saved_tracks(offset=offset_index)
    for song in songs['items']:
        
        #join track ids to a string for audio_features function
        track_list += song['track']['id'] +','
        
        #get the time when the song was added
        added_ts_list.append(song['added_at'])
        
        #get the title of the song
        title_list.append(song['track']['name'])
        
        #get all the artists in the song
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ', '
        artist_list.append(artists_name[:-2])
        
    #get the track features and append into a dataframe
    track_features = sp.audio_features(track_list[:-1])
    df_temp = pd.DataFrame(track_features)
    df_saved_tracks = df_saved_tracks.append(df_temp)
    track_list = ''
    if songs['next'] == None:
        # no more songs in playlist
        more_songs = False
    else:
        # get the next n songs
        offset_index += songs['limit']
        
#include title and artists of a song
df_saved_tracks['added_at'] = added_ts_list
df_saved_tracks['song_title'] = title_list
df_saved_tracks['artists'] = artist_list

In [17]:
df_saved_tracks

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,added_at,song_title,artists
0,0.472,0.671,4,-4.649,1,0.0290,0.01290,0.000000,0.1590,0.0681,...,audio_features,77loZpT5Y5PRP1S451P9Yz,spotify:track:77loZpT5Y5PRP1S451P9Yz,https://api.spotify.com/v1/tracks/77loZpT5Y5PR...,https://api.spotify.com/v1/audio-analysis/77lo...,232800,4,2020-04-17T12:14:53Z,The Reason,Hoobastank
1,0.462,0.745,6,-4.278,0,0.0497,0.00937,0.009640,0.2220,0.3510,...,audio_features,29TH6WPBjWj7pqIRTrhDCQ,spotify:track:29TH6WPBjWj7pqIRTrhDCQ,https://api.spotify.com/v1/tracks/29TH6WPBjWj7...,https://api.spotify.com/v1/audio-analysis/29TH...,147238,4,2020-04-14T19:08:05Z,Sweaters,"just a fake, ivan B"
2,0.458,0.696,11,-4.742,1,0.0402,0.28500,0.000000,0.1230,0.4180,...,audio_features,4B2kkxg3wKSTZw5JPaUtzQ,spotify:track:4B2kkxg3wKSTZw5JPaUtzQ,https://api.spotify.com/v1/tracks/4B2kkxg3wKST...,https://api.spotify.com/v1/audio-analysis/4B2k...,207069,4,2020-04-09T12:47:32Z,On & On,"Cartoon, Daniel Levi"
3,0.751,0.344,6,-7.712,1,0.0379,0.52800,0.000002,0.1130,0.4710,...,audio_features,3FTRSa9mWB3kKx2jkhUAN3,spotify:track:3FTRSa9mWB3kKx2jkhUAN3,https://api.spotify.com/v1/tracks/3FTRSa9mWB3k...,https://api.spotify.com/v1/audio-analysis/3FTR...,235932,3,2020-04-09T12:37:05Z,Tum Jab Paas,Prateek Kuhad
4,0.362,0.763,3,-8.514,1,0.0589,0.01500,0.004810,0.2370,0.4310,...,audio_features,1gWtHDe403RuyZqhvSRcv8,spotify:track:1gWtHDe403RuyZqhvSRcv8,https://api.spotify.com/v1/tracks/1gWtHDe403Ru...,https://api.spotify.com/v1/audio-analysis/1gWt...,216886,4,2020-04-09T12:31:52Z,Tune Kaha,Prateek Kuhad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,0.356,0.470,5,-8.929,1,0.0278,0.51800,0.000010,0.0807,0.1670,...,audio_features,0hc0Eu0WuS3LtZae4WgTaZ,spotify:track:0hc0Eu0WuS3LtZae4WgTaZ,https://api.spotify.com/v1/tracks/0hc0Eu0WuS3L...,https://api.spotify.com/v1/audio-analysis/0hc0...,316500,4,2019-07-28T10:04:37Z,"Tum Ho (From ""Rockstar"")","Mohit Chauhan, Suzanne D'Mello"
14,0.406,0.791,11,-3.804,1,0.0434,0.16900,0.000000,0.1670,0.4930,...,audio_features,6QiBNpMK7xlZFnnoEdIPCr,spotify:track:6QiBNpMK7xlZFnnoEdIPCr,https://api.spotify.com/v1/tracks/6QiBNpMK7xlZ...,https://api.spotify.com/v1/audio-analysis/6QiB...,189467,4,2019-07-22T16:29:54Z,Millionaires,The Script
15,0.286,0.769,6,-4.609,0,0.1260,0.02760,0.000000,0.1320,0.2010,...,audio_features,2vujPL9jOWisAKM4yCrTPL,spotify:track:2vujPL9jOWisAKM4yCrTPL,https://api.spotify.com/v1/tracks/2vujPL9jOWis...,https://api.spotify.com/v1/audio-analysis/2vuj...,225897,4,2019-07-07T21:06:29Z,Horns (Arc North Remix),"Arc North, Bryce Fox"
16,0.406,0.633,0,-6.989,1,0.0379,0.51300,0.000000,0.1160,0.3440,...,audio_features,6YycppSAeGtSvwmu8qCcr2,spotify:track:6YycppSAeGtSvwmu8qCcr2,https://api.spotify.com/v1/tracks/6YycppSAeGtS...,https://api.spotify.com/v1/audio-analysis/6Yyc...,397027,4,2019-07-05T03:22:50Z,In Dino,"Pritam, Soham"


In [27]:
df_saved_tracks.to_excel('saved_tracks.xlsx')